# Proyecto: Siniestro viales

La empresa bogotana de seguros SSeguro esta buscando obtener informacion sobre los datos obtenidos en siniestros viales. Se busca lograr esto cargando la informacion a una base de datos de `sqlite` y haciendo consultas `SQL` para obtener la mayor cantidad de informacion sobre los datos.

### Metas

En general, se busca crear una base de datos local para poder ejecutar consultas de `SQL`. Ademas se busca generar consultas que extraigan informacion que permitan identificar caracteristicas de la muestra, es decir, obtener conteos y promedios de la muestra con respecto a diferentes caracteristicas. En mas detalle, se espera obtener informacion como:

* El promedio de edad por sexo, estado y condicion.
* La cantidad de personas por sexo y la cantidad de sevicios que entran en fuga.

### Planeacion

La empresa nos dio acceso a una base de datos de github en formato csv, el primer paso a seguir es importar los datos, es decir, hacer uso de pandas para descargar los datos en un DataFrame. Con esto se procede a exportar los datos a una base de datos local en un archivo `.db`. Finalmente, se hacen las consultas *SQL* necesarias para obtener la informacion de los datos.

## Importacion de paquetes y carga de datos

Inicialmente se buscaba usar la funcion de pandas `to_sql` con conexion a un *engine* de `SQLAlchemy`, pero los intentos no fueron eficientes en velocidad, por lo tanto, se decidio usar un metodo local y manual. Esto quiere decir que se uso el paquete `sqlite3` y se implemento una carga de datos usando un bucle **for** insertando individualmente cada fila de los datos. Se intento una insercion multiple pero no hubo mayor eficiencia, por lo tanto se conservo insercion individual por menor cantidad de codigo necesitado. Teniendo en cuenta esto, podemos empezar con la importacion de los paquetes necesarios:

In [1]:
import pandas as pd
import sqlite3 as db

Ahora, creamos una conexion a la base de datos, conectando a una existente o creandose automaticamente un archivo `.db` si no existe previamente:

In [2]:
con = db.connect("SiniestrosViales.db")
cur = con.cursor()

Bajo esta conexion creamos la tabla `Siniestros`, la cual consta de las siguientes 16 columnas:
* fecha
* gravedad
* clase_siniestro
* choque
* objeto_fijo
* localidad
* lugar
* causa
* vehiculo
* servicio
* modalidad
* fuga
* condicion
* estado
* edad
* sexo

In [3]:
cur.execute("""CREATE TABLE Siniestros (
            fecha,
            gravedad INTEGER,
            clase_siniestro INTEGER,
            choque INTEGER,
            objeto_fijo,
            localidad INTEGER,
            lugar INTEGER,
            causa INTEGER,
            vehiculo INTEGER,
            servicio INTEGER,
            modalidad INTEGER,
            fuga,
            condicion,
            estado,
            edad INTEGER,
            sexo)
            """)

y definimos las siguientes funciones encargadas de insertar una columna y todo el dataframe respectivamente:

In [6]:
def insert_row(row):
    row = ' '.join(["'"+str(elem)+"', " for elem in row])
    query = f"""
    INSERT INTO Siniestros(fecha, gravedad, clase_siniestro, choque, objeto_fijo, localidad, lugar, causa, vehiculo, servicio, modalidad, fuga, condicion, estado, edad, sexo)
    VALUES
	    ({row[:-2]});
    """
    cur.execute(query)
    con.commit()

def insert_df(df):
    for i in range(len(df)):
        insert_row(df.iloc[i].array)
    print("Ejecucion exitosa")

Finalmente, procedemos a importar los datos en el dataframe `df` y ejecutamos la insercion de los datos:

In [5]:
df = pd.read_csv(r"https://raw.githubusercontent.com/JuanAGuzman/Siniestros-Viales/main/siniestros_viales.csv")
df.drop(['Unnamed: 0', 'CODIGO_ACCIDENTE', 'DIRECCION', 'CODIGO_ACCIDENTADO', 'VEHICULO'], axis=1, inplace=True)

In [7]:
insert_df(df)

Ejecucion exitosa


Se uso la funcion `print` para hacer seguimiento cada 10000 datos insertados.

## Exploracion de los datos

Habiendo creado la base de datos, podemos empezar con la consultas para extraer informacion de los datos. Para facilitar las consultas se va a trabajar con funciones que retornen los `query`, los cuales pueden ser evaluados directamente de la siguiente forma:

In [8]:
def distintos_valores(variable):
    query = f"""
    SELECT DISTINCT
        {variable},
        COUNT(*) as conteo
    FROM
        Siniestros
    GROUP BY
        {variable}
    """
    return query

In [9]:
res = cur.execute(distintos_valores('gravedad'))
res.fetchall()

[(1, 8646), (2, 191111), (3, 302210)]

Pero con pandas podemos obtener una visualizacion mas agradable y comprensible de la siguiente forma:

In [10]:
pd.read_sql(distintos_valores('sexo'), con)

,sexo,conteo
0,FEMENINO,75936
1,MASCULINO,414323
2,SIN INFORMACION,11708


Por lo tanto, se va a usar pandas para ejecutar dichas consultas.

Con la funcion `distintos_valores` definida, podemos identificar los diferentes valores que tomas las variables y la cantidad de cada valor:

In [11]:
pd.read_sql(distintos_valores('condicion'), con)

,condicion,conteo
0,CICLISTA,16915
1,CONDUCTOR,361433
2,MOTOCICLISTA,66131
3,PASAJERO/ACOMPAÑANTE,34041
4,PEATON,23447


In [12]:
pd.read_sql(distintos_valores('estado'), con)

,estado,conteo
0,HERIDO,114199
1,ILESO,384213
2,MUERTO,3555


In [13]:
pd.read_sql(distintos_valores('servicio'), con)

,servicio,conteo
0,0,48055
1,1,9513
2,2,188444
3,3,255859
4,4,96


In [14]:
pd.read_sql(distintos_valores('fuga'), con)

,fuga,conteo
0,N,493610
1,S,8357


In [15]:
pd.read_sql(distintos_valores('edad'), con)

,edad,conteo
0,0,324
1,1,352
2,2,366
3,3,348
4,4,379
...,...,...
109,117,13
110,118,328
111,119,1
112,120,6


In [16]:
pd.read_sql(distintos_valores('localidad'), con)

,localidad,conteo
0,1,47826
1,2,28071
2,3,14365
3,4,14426
4,5,10559
5,6,14712
6,7,25426
7,8,58140
8,9,38953
9,10,50294


In [17]:
pd.read_sql(distintos_valores('lugar'), con)

,lugar,conteo
0,1,378602
1,2,106047
2,3,205
3,4,3266
4,5,1474
5,6,739
6,7,5238
7,8,940
8,9,2
9,10,4498


Esto es de gran utilidad debido a que nos da una idea del contenido de los datos y abre camino a realizar nuevas distintas consultas.

#### Resultados

Podemos empezar nuestras consultas mas avanzadas definiendo las siguientes funciones query:

In [18]:
def implicados_edad_sexo(inf, sup):
    query = f"""
    SELECT
        sexo,
        COUNT(*) AS conteo
    FROM
        Siniestros
    WHERE
        edad BETWEEN {inf} AND {sup}
    GROUP BY
        sexo
    """
    return query

In [19]:
def promedio_edad(variable):
    query = f"""
    SELECT
        {variable},
        AVG(edad) AS edad_promedio
    FROM
        Siniestros
    GROUP BY
        {variable}
    """
    return query

In [20]:
def estado_sexo():
    query = """
    SELECT
        sexo,
        estado,
        COUNT(*) AS conteo
    FROM
        Siniestros
    GROUP BY
        sexo,
        estado
    """
    return query

In [21]:
def enfuga_sexo():
    query = """
    SELECT
        sexo,
        fuga,
        COUNT(*) AS conteo
    FROM
        Siniestros
    GROUP BY
        sexo,
        fuga
    """
    return query

In [22]:
def enfuga_servicio():
    query = """
    SELECT
        servicio,
        fuga,
        COUNT(*) AS conteo
    FROM
        Siniestros
    GROUP BY
        servicio,
        fuga
    """
    return query

A partir de estas podemos obtener los diferentes valores de consulta, repasemos una por una:

Con la funcion `implicados_edad_sexo` podemos obtener la cantidad de personas involucradas en siniestros por sexo en un rango de edad:

In [23]:
pd.read_sql(implicados_edad_sexo(18, 35), con)

,sexo,conteo
0,FEMENINO,34410
1,MASCULINO,199387
2,SIN INFORMACION,1001


Por otro lado, la funcion `promedio_edad` permite obtener la edad promedio para la variable indicada, veamos unos ejemplos:

In [24]:
pd.read_sql(promedio_edad('sexo'), con)

,sexo,edad_promedio
0,FEMENINO,37.015658
1,MASCULINO,37.672031
2,SIN INFORMACION,7.160147


In [25]:
pd.read_sql(promedio_edad('condicion'), con)

,condicion,edad_promedio
0,CICLISTA,31.937984
1,CONDUCTOR,38.657367
2,MOTOCICLISTA,29.344422
3,PASAJERO/ACOMPAÑANTE,33.613466
4,PEATON,38.638205


In [26]:
pd.read_sql(promedio_edad('estado'), con)

,estado,edad_promedio
0,HERIDO,33.255808
1,ILESO,37.919248
2,MUERTO,38.310267


Observamos que el promedio oscila entre los 29 y los 39 años, esto nos indica una mayor poblacion joven al volante.

Del estado tambien podemos identificar la cantidad de personas por sexo en cada estado usando la funcion `estado_sexo`:

In [27]:
pd.read_sql(estado_sexo(), con)

,sexo,estado,conteo
0,FEMENINO,HERIDO,39123
1,FEMENINO,ILESO,36147
2,FEMENINO,MUERTO,666
3,MASCULINO,HERIDO,74126
4,MASCULINO,ILESO,337536
5,MASCULINO,MUERTO,2661
6,SIN INFORMACION,HERIDO,950
7,SIN INFORMACION,ILESO,10530
8,SIN INFORMACION,MUERTO,228


Observamos como es menor la proporcion de heridos y muertos con respecto a los ilesos para los hombres que para las mujeres.

Finalmente, podemos identificar la catidad de personas que entran en fuga para cada sexo y tipo de servicio:

In [28]:
pd.read_sql(enfuga_sexo(), con)

,sexo,fuga,conteo
0,FEMENINO,N,75192
1,FEMENINO,S,744
2,MASCULINO,N,413339
3,MASCULINO,S,984
4,SIN INFORMACION,N,5079
5,SIN INFORMACION,S,6629


In [29]:
pd.read_sql(enfuga_servicio(), con)

,servicio,fuga,conteo
0,0,N,39698
1,0,S,8357
2,1,N,9513
3,2,N,188444
4,3,N,255859
5,4,N,96


Notamos que hay una mayor cantidad de hombres que entran en fuga, pero la proporcion es mauor para las mujeres, de igual forma identificamos que unicamente se registran fugas de servicios del tipo 1.

Finalmente, despues de realizar todas las consultas podemos cerrar la conexion con la base de datos.

In [7]:
con.close()

## Conclusiones

Toda la informacion obtenida es de gran utilidad debido a que esta puede ser usada para determinar el publico objetivo de la empresa y en como se van a destinar los recursos, por ejemplo identificando que los promedios de edad podemos identificar que tipo de medios y compañas utilizar para publicitar la empresa.
Esto nos indica la importancia de trabajar con bases de datos, dado que usando el lenguaje de `SQL` podemos generar consultas de manera simple que extraigan informacion valiosa de los datos.

Realizado por Juan Andres Guzman para el Modulo 3 'Big Data' del diplomado modular 'Machine learning and Data science' ofrecido por la universidad Nacional de Colombia.